# Predicción de postulación para un aviso

In [1]:
import numpy as np  
import pandas as pd
import gc
import datetime
import re
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder

In [2]:
SAMPLE_SIZE = 1000000

## Carga y limpieza de datos / Feature Engineering

### Postulantes

In [3]:
# cargo postulantes
df_postulantes1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])
df_postulantes2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])
df_postulantes3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])

df_postulantes = df_postulantes1.append(df_postulantes2).append(df_postulantes3)

del df_postulantes1
del df_postulantes2
del df_postulantes3

df_postulantes.drop_duplicates(['idpostulante'], keep='first', inplace=True)

df_postulantes.shape

(504407, 3)

In [4]:
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# guardamos los codificadores (label => numero y visceversa) en un diccionario
label_encoders = {}

In [5]:
# limpieza de datos de fecha de nacimiento
df_postulantes['fechanacimiento'] = pd.to_datetime(df_postulantes['fechanacimiento'], errors='coerce')

df_postulantes['edad'] = datetime.datetime.now().year - df_postulantes['fechanacimiento'].dt.year
df_postulantes['edad'] = df_postulantes['edad'].fillna(0)

df_postulantes = df_postulantes.drop(['fechanacimiento'], axis=1)

df_postulantes = df_postulantes.loc[(df_postulantes['sexo'] == 'FEM') | (df_postulantes['sexo'] == 'MASC') | (df_postulantes['sexo'] == 'NO_DECLARA')]

# convierto variables categóricas a numéricas
label_encoders['sexo'] = LabelEncoder().fit(['FEM', 'MASC', 'NO_DECLARA'])
df_postulantes['sexo'] = label_encoders['sexo'].transform(df_postulantes['sexo'])

print(df_postulantes.shape)
print(df_postulantes.head())

(504400, 3)
  idpostulante  sexo  edad
0         NM5M     0  48.0
1         5awk     0  56.0
2         ZaO5     0  40.0
3         NdJl     1  49.0
4         eo2p     1  37.0


### Educacion

In [6]:
# cargo educacion de los estudiantes
df_edu1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_1_postulantes_educacion.csv')
df_edu2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
df_edu3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv')

df_edu = df_edu1.append(df_edu2).append(df_edu3)

del df_edu1
del df_edu2
del df_edu3
gc.collect()

df_edu.drop_duplicates(['idpostulante'], keep='first', inplace=True)

print(df_edu.shape)
print(df_edu.head())

(447909, 3)
  idpostulante         nombre    estado
0         NdJl       Posgrado  En Curso
1         8BkL  Universitario  En Curso
2         1d2B  Universitario  En Curso
3         NPBx  Universitario  En Curso
5         Ym2X           Otro  En Curso


In [7]:
# renombro columnas para no confundirlas luego de mergear
df_edu = df_edu.rename(columns={'nombre':'nombre_edu', 'estado': 'estado_edu'});

In [8]:
# convierto variables categóricas a numéricas
label_encoders['nombre_edu'] = LabelEncoder().fit(df_edu['nombre_edu'])
label_encoders['estado_edu'] = LabelEncoder().fit(df_edu['estado_edu'])

df_edu['nombre_edu'] = label_encoders['nombre_edu'].transform(df_edu['nombre_edu'])
df_edu['estado_edu'] = label_encoders['estado_edu'].transform(df_edu['estado_edu'])

df_edu.head()

,idpostulante,nombre_edu,estado_edu
0,NdJl,3,1
1,8BkL,6,1
2,1d2B,6,1
3,NPBx,6,1
5,Ym2X,2,1


In [9]:
df_posts_edu = df_postulantes.merge(df_edu, on='idpostulante', how='left')

df_posts_edu['nombre_edu'] = df_posts_edu['nombre_edu'].fillna(-1)
df_posts_edu['estado_edu'] = df_posts_edu['estado_edu'].fillna(-1)

# me quedo con el de mayor educacion registro para cada postulante
df_posts_edu.sort_values(by='nombre_edu', ascending=False)

df_posts_edu.drop_duplicates(subset = "idpostulante",keep= "first", inplace=True)

print(df_posts_edu.shape)
print(df_posts_edu.head())

del df_edu
del df_postulantes
gc.collect()

(504400, 5)
  idpostulante  sexo  edad  nombre_edu  estado_edu
0         NM5M     0  48.0         4.0         2.0
1         5awk     0  56.0         6.0         2.0
2         ZaO5     0  40.0         5.0         2.0
3         NdJl     1  49.0         3.0         1.0
4         eo2p     1  37.0         4.0         2.0


95

### Postulaciones

In [10]:
# cargo postulaciones
df_postulaciones1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_4_postulaciones.csv')
df_postulaciones2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')

df_postulaciones = df_postulaciones1.append(df_postulaciones2)

del df_postulaciones1
del df_postulaciones2
gc.collect()

df_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], keep='first', inplace=True)

print(df_postulaciones.shape)
print(df_postulaciones.head())

(6603752, 3)
      idaviso idpostulante     fechapostulacion
0  1112257047         NM5M  2018-01-15 16:22:34
1  1111920714         NM5M  2018-02-06 09:04:50
2  1112346945         NM5M  2018-02-22 09:04:47
3  1112345547         NM5M  2018-02-22 09:04:59
4  1112237522         5awk  2018-01-25 18:55:03


In [11]:
# no necesitamos la fecha de postulacion
df_postulaciones.drop(['fechapostulacion'], axis=1, inplace=True)
df_postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


### Avisos

In [12]:
# cargo avisos
df_avisos1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_6_avisos_detalle.csv')
df_avisos2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos4 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

df_avisos = df_avisos1.append(df_avisos2).append(df_avisos3).append(df_avisos4)

del df_avisos1
del df_avisos2
del df_avisos3
del df_avisos4
gc.collect()

df_avisos = df_avisos.drop_duplicates(['idaviso'], keep='first')

print(df_avisos.shape)
print(df_avisos.head())

(25288, 11)
      idaviso  idpais                                         titulo  \
0     8725750       1               VENDEDOR/A PROVINCIA DE SANTA FE   
1    17903700       1                                     Enfermeras   
2  1000150677       1                                 Chofer de taxi   
3  1000610287       1  CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA   
4  1000872556       1  Operarios de Planta - Rubro Electrodomésticos   

                                         descripcion        nombre_zona  \
0  <p><strong><strong>Empresa:</strong></strong> ...  Gran Buenos Aires   
1  <p>Solicitamos para importante cadena de farma...  Gran Buenos Aires   
2  <p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...    Capital Federal   
3  <p><strong>Somos una empresa multinacional que...  Gran Buenos Aires   
4  <p><strong>OPERARIOS DE PLANTA</strong></p><p>...  Gran Buenos Aires   

  ciudad       mapacalle tipo_de_trabajo         nivel_laboral nombre_area  \
0    NaN             NaN  

#### Avisos online

In [13]:
# cargo avisos
df_avisos_online1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_5_avisos_online.csv')
df_avisos_online2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv')

df_avisos_online = df_avisos_online1.append(df_avisos_online2)

del df_avisos_online1
del df_avisos_online2
gc.collect()

df_avisos_online = df_avisos_online.drop_duplicates(['idaviso'], keep='first')
df_avisos_online['online'] = 1

print(df_avisos_online.shape)
print(df_avisos_online.head())

(9430, 2)
      idaviso  online
0  1112355872       1
1  1112335374       1
2  1112374842       1
3  1111984070       1
4  1111822480       1


In [14]:
df_avisos = df_avisos.merge(df_avisos_online, how='left', on='idaviso')

del df_avisos_online
gc.collect()

73

In [15]:
df_avisos = df_avisos.drop(['mapacalle'], axis=1)

In [16]:
# limpieza de NaN, nan, None, etc.
df_avisos['ciudad'] = df_avisos['ciudad'].fillna('None')
df_avisos['titulo'] = df_avisos['titulo'].fillna('None')
df_avisos['descripcion'] = df_avisos['descripcion'].fillna('None')
df_avisos['denominacion_empresa'] = df_avisos['denominacion_empresa'].fillna('None')
df_avisos['nivel_laboral'] = df_avisos['nivel_laboral'].fillna('None')

df_avisos['online'] = df_avisos['online'].fillna(0)

In [17]:
# convierto variables categóricas a numéricas
label_encoders['nombre_zona'] = LabelEncoder().fit(df_avisos['nombre_zona'])
label_encoders['ciudad'] = LabelEncoder().fit(df_avisos['ciudad'])
label_encoders['tipo_de_trabajo'] = LabelEncoder().fit(df_avisos['tipo_de_trabajo'])
label_encoders['nivel_laboral'] = LabelEncoder().fit(df_avisos['nivel_laboral'])
label_encoders['nombre_area'] = LabelEncoder().fit(df_avisos['nombre_area'])
label_encoders['denominacion_empresa'] = LabelEncoder().fit(df_avisos['denominacion_empresa'])

df_avisos['nombre_zona'] = label_encoders['nombre_zona'].transform(df_avisos['nombre_zona'])
df_avisos['ciudad'] = label_encoders['ciudad'].transform(df_avisos['ciudad'])
df_avisos['tipo_de_trabajo'] = label_encoders['tipo_de_trabajo'].transform(df_avisos['tipo_de_trabajo'])
df_avisos['nivel_laboral'] = label_encoders['nivel_laboral'].transform(df_avisos['nivel_laboral'])
df_avisos['nombre_area'] = label_encoders['nombre_area'].transform(df_avisos['nombre_area'])
df_avisos['denominacion_empresa'] = label_encoders['denominacion_empresa'].transform(df_avisos['denominacion_empresa'])

In [18]:
df_avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,online
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,7,30,1,5,30,4005,0.0
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,7,30,1,5,158,1640,1.0
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,1,30,1,5,181,1561,0.0
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,7,30,1,5,181,4119,1.0
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,7,30,1,5,143,1267,0.0


#### Trabajando el texto/titulo de los avisos

In [19]:
regex_limpiar_html_tags = re.compile('<.*?>')
def limpiar_html(strhtml):
    return re.sub(regex_limpiar_html_tags, '', strhtml)

def regularizar_texto(linea):
    return limpiar_html(linea)\
                        .lower()\
                        .replace('á', 'a')\
                        .replace('é', 'e')\
                        .replace('í', 'i')\
                        .replace('ó', 'o')\
                        .replace('ú', 'u')\
                        .replace('\t', '')\
                        .replace('\n', '')\
                        .replace('\r', '')
                        
vregularizar_texto = np.vectorize(regularizar_texto)

columnas_terminos = {
    'ingenieria': ['ingeniero', 'ingeniera', 'ingenieria'],
    'software': ['javascript', 'java', 'html', 'css', 'c#', '.net', 'android', 'ios', 'php', 'c++'],
    'lunes_a_viernes': ['lunes a viernes', 'lun a vier', 'lun a vie'],
    'requiere_titulo': ['titulo secundario', 'titulo terciario', 'titulo universitario', 'secundario completo', 'estudios completo', 'universitarios completo'],
    'marketing': ['marketing', 'telemarketer', 'telemarketing', 'marketer', 'media manager', 'callcenter', 'call center'],
    'capacitacion': ['capacitacion'],
    'idioma_ingles': ['idioma ingles', 'manejo de ingles', 'clases de ingles', 'ingles excluyente', 'ingles requerido'],
    'multinacional': ['multinacional'],
    'internacional': ['internacional'],
    'atencion_al_cliente': ['atencion al cliente', 'call center', 'callcenter', 'soporte tecnico', 'area de soporte', 'tareas de soporte'],
    'turismo': ['turismo'],
    'zona_puerto_madero': ['puerto madero'],
    'zona_centro': ['microcentro', 'tribunales'],
    'experiencia_previa': ['experiencia previa', 'experiencias anteriores', 'años de experiencia'],
    'obra_social': ['obra social', 'osde', 'swiss medical', 'galeno', 'wh hope', 'grupo familiar', 'cobertura medica', 'pre paga', 'prepaga'],
    'puesto_gerencia': ['gerente', 'gerenta', 'gerencia'],
    'requisitos_excluyentes': ['excluyente'],
    'retail': ['hipermercado', 'supermercado', 'cadena', 'franquicia', 'fravega', 'retail'],
}

def tiene_termino(texto1, texto2, terminos):
    for t in terminos:
        if t in texto1 or t in texto2:
            return 1
    return 0
def vtiene_termino(serie1, serie2, terminos):
    if len(serie1) != len(serie2):
        raise ValueError('series de distinto largo')
    s = []
    for i in range(0, len(serie1)):
        s.append(tiene_termino(serie1.iloc[i], serie2.iloc[i], terminos))
    return pd.Series(s)

In [20]:
df_avisos['titulo'] = vregularizar_texto(df_avisos['titulo'])
df_avisos['descripcion'] = vregularizar_texto(df_avisos['descripcion'])

In [21]:
# optimizable, podríamos aplicar para cada fila todas las columnas del diccionario
# como está ahora hace k*n con k=|columnas_terminos| y n=|df_avisos|
for col, terminos in columnas_terminos.items():
    df_avisos[col] = vtiene_termino(df_avisos['titulo'], df_avisos['descripcion'], terminos)

In [22]:
for col in columnas_terminos:
    print("col = %s" % col)
    print(df_avisos[col].value_counts())

col = ingenieria
0    22189
1     3099
Name: ingenieria, dtype: int64
col = software
1    15564
0     9724
Name: software, dtype: int64
col = lunes_a_viernes
0    18738
1     6550
Name: lunes_a_viernes, dtype: int64
col = requiere_titulo
0    21691
1     3597
Name: requiere_titulo, dtype: int64
col = marketing
0    22784
1     2504
Name: marketing, dtype: int64
col = capacitacion
0    21815
1     3473
Name: capacitacion, dtype: int64
col = idioma_ingles
0    23684
1     1604
Name: idioma_ingles, dtype: int64
col = multinacional
0    23242
1     2046
Name: multinacional, dtype: int64
col = internacional
0    24150
1     1138
Name: internacional, dtype: int64
col = atencion_al_cliente
0    22994
1     2294
Name: atencion_al_cliente, dtype: int64
col = turismo
0    25114
1      174
Name: turismo, dtype: int64
col = zona_puerto_madero
0    24996
1      292
Name: zona_puerto_madero, dtype: int64
col = zona_centro
0    24154
1     1134
Name: zona_centro, dtype: int64
col = experiencia_previa

In [23]:
df_avisos.drop(['titulo', 'descripcion'], axis=1, inplace=True)

In [24]:
gc.collect()

25

### Vinculamos postulantes y avisos

In [25]:
df_postulaciones_merge = df_postulaciones.sample(SAMPLE_SIZE)

# merge de todos los datos
df_general = df_posts_edu.merge(df_postulaciones_merge, on='idpostulante').merge(df_avisos, on='idaviso')

del df_postulaciones_merge
gc.collect()

print(df_general.shape)
print(df_general.head())

(937237, 32)
  idpostulante  sexo  edad  nombre_edu  estado_edu     idaviso  idpais  \
0         5awk     0  56.0         6.0         2.0  1112305358       1   
1       5mBe0Z     1  45.0         4.0         2.0  1112305358       1   
2       6bMvEr     1  42.0         6.0         2.0  1112305358       1   
3       EwKNO6     1  41.0         6.0         2.0  1112305358       1   
4       5Mwjak     1  39.0         4.0         2.0  1112305358       1   

   nombre_zona  ciudad  tipo_de_trabajo   ...    internacional  \
0            7      30                1   ...                0   
1            7      30                1   ...                0   
2            7      30                1   ...                0   
3            7      30                1   ...                0   
4            7      30                1   ...                0   

   atencion_al_cliente  turismo  zona_puerto_madero  zona_centro  \
0                    0        0                   0            0   
1        

### Vistas

In [26]:
# cargo avisos
df_vistas1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_3_vistas.csv', parse_dates=['timestamp'])
df_vistas2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_3_vistas.csv', parse_dates=['timestamp'])
df_vistas3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_3_vistas.csv', parse_dates=['timestamp'])

df_vistas = df_vistas1.append(df_vistas2).append(df_vistas3)

del df_vistas1
del df_vistas2
del df_vistas3
gc.collect()

df_vistas = df_vistas.rename(columns={'idAviso':'idaviso'})
df_vistas = df_vistas.drop_duplicates(['idpostulante', 'idaviso'], keep='first')
gc.collect()

print(df_vistas.shape)
print(df_vistas.head())

(8597915, 3)
      idaviso               timestamp idpostulante
0  1111780242 2018-02-23 18:38:13.187      YjVJQ6Z
1  1112263876 2018-02-23 18:38:14.296      BmVpYoR
2  1112327963 2018-02-23 18:38:14.329      wVkBzZd
3  1112318643 2018-02-23 18:38:17.921      OqmP9pv
4  1111903673 2018-02-23 18:38:18.973      DrpbXDP


In [27]:
df_vistas.head()

,idaviso,timestamp,idpostulante
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP


In [28]:
#vistas_por_aviso = df_vistas.groupby('idaviso')\
#                             .agg({'timestamp': 'count'})\
#                             .reset_index()\
#                             .rename(columns={'timestamp': 'cantvistas'})

In [29]:
#vistas_por_aviso.head()
#df_vistas.drop('timestamp', )
df_vistas['visto'] = 1

In [30]:
#df_general = df_general.merge(vistas_por_aviso, on=['idaviso'])

In [31]:
#df_general.loc[df_general['cantvistas'] > 1]

df_general

,idpostulante,sexo,edad,nombre_edu,estado_edu,idaviso,idpais,nombre_zona,ciudad,tipo_de_trabajo,...,internacional,atencion_al_cliente,turismo,zona_puerto_madero,zona_centro,experiencia_previa,obra_social,puesto_gerencia,requisitos_excluyentes,retail
0,5awk,0,56.0,6.0,2.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
1,5mBe0Z,1,45.0,4.0,2.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
2,6bMvEr,1,42.0,6.0,2.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
3,EwKNO6,1,41.0,6.0,2.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
4,5Mwjak,1,39.0,4.0,2.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
5,8BAQPz,1,34.0,6.0,1.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
6,eaQLGE,0,33.0,6.0,2.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
7,YOzB6P,0,35.0,6.0,1.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
8,5xkQvG,1,57.0,5.0,2.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
9,YRqOEP,1,35.0,6.0,1.0,1112305358,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0


## Preparación de datos para entrenamiento y predicción

#### Generación de postulaciones

In [32]:
df_general['sepostulo'] = 1

#### Generación de "no" postulaciones

In [33]:
sample = SAMPLE_SIZE

In [34]:
df_postulantes_sample = df_posts_edu.sample(sample, replace=True).reset_index().drop("index",1)
df_avisos_sample = df_avisos.sample(sample, replace=True).reset_index().drop("index",1)

print(df_postulantes_sample.shape)
print(df_avisos_sample.shape)

(1000000, 5)
(1000000, 27)


In [35]:
df_no_postulaciones = df_postulantes_sample.join(df_avisos_sample)

del df_postulantes_sample
del df_avisos_sample
gc.collect()

df_no_postulaciones = df_no_postulaciones.merge(df_postulaciones, on=["idaviso","idpostulante"], how="left")
df_no_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], keep='first', inplace=True)
print(df_no_postulaciones.shape)

(999955, 32)


In [36]:
del df_postulaciones
gc.collect()

54

In [37]:
df_no_postulaciones['sepostulo'] = 0;
df_no_postulaciones.head(5)

,idpostulante,sexo,edad,nombre_edu,estado_edu,idaviso,idpais,nombre_zona,ciudad,tipo_de_trabajo,...,atencion_al_cliente,turismo,zona_puerto_madero,zona_centro,experiencia_previa,obra_social,puesto_gerencia,requisitos_excluyentes,retail,sepostulo
0,qevKMJk,1,29.0,4.0,2.0,1112382031,1,7,30,1,...,0,0,0,0,0,0,0,0,0,0
1,zvaNXBk,1,23.0,4.0,2.0,1112241778,1,7,30,1,...,0,0,0,1,0,0,0,0,0,0
2,mzdNbl3,0,19.0,2.0,1.0,1112403117,1,7,30,1,...,0,0,0,0,0,0,0,1,0,0
3,BmZ9kZa,1,29.0,6.0,2.0,1112458435,1,7,30,1,...,0,0,0,0,1,0,0,0,0,0
4,zv8xLLJ,0,25.0,6.0,1.0,1112030964,1,7,30,1,...,0,0,0,0,1,0,1,1,0,0


In [38]:
print(df_general.shape)
df_general = df_general.append(df_no_postulaciones)
print(df_general.shape)

(937237, 33)
(1937192, 33)


In [39]:
df_general = df_general.merge(df_vistas, on=['idaviso', 'idpostulante'], how='left')
df_general['visto'] = df_general['visto'].fillna(0)

In [40]:
# no estoy cargando los datos que cargaría en el caso real, así que no los necesito
del df_vistas
del df_posts_edu
del df_avisos

gc.collect()

153

In [41]:
#df_general = df_general.merge(vistas_por_aviso, on=['idaviso'])

In [42]:
df_general = shuffle(df_general, random_state=13).reset_index()

gc.collect()

32

In [43]:
offset = int(df_general.shape[0] * 0.8)

df_general_entrenamiento = df_general.loc[:offset]
df_general_test = df_general.loc[offset:]

In [44]:
del df_general
gc.collect()

27

In [45]:
df_general_entrenamiento.columns

Index(['index', 'idpostulante', 'sexo', 'edad', 'nombre_edu', 'estado_edu',
       'idaviso', 'idpais', 'nombre_zona', 'ciudad', 'tipo_de_trabajo',
       'nivel_laboral', 'nombre_area', 'denominacion_empresa', 'online',
       'ingenieria', 'software', 'lunes_a_viernes', 'requiere_titulo',
       'marketing', 'capacitacion', 'idioma_ingles', 'multinacional',
       'internacional', 'atencion_al_cliente', 'turismo', 'zona_puerto_madero',
       'zona_centro', 'experiencia_previa', 'obra_social', 'puesto_gerencia',
       'requisitos_excluyentes', 'retail', 'sepostulo', 'timestamp', 'visto'],
      dtype='object')

## Ejecución del algoritmo de ML

In [46]:
#columnas_datos = ['sexo', 'edad', 'nombre_edu', 'estado_edu', 'idpais', 'nombre_zona', 'ciudad', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa', 'online', 'cantvistas'] + list(columnas_terminos.keys())
columnas_datos = ['sexo', 'edad', 'nombre_edu', 'estado_edu', 'idpais', 'nombre_zona', 'ciudad', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa', 'online', 'visto'] + list(columnas_terminos.keys())
columnas_target = ['sepostulo']

In [47]:
print(df_general_entrenamiento.shape)
print(df_general_test.shape)

(1549754, 36)
(387439, 36)


### Decision Tree

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
#params = { 'max_depth': 150, 'random_state': 29 }

#treeclassifier = DecisionTreeClassifier(**params)
#treeclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target])

#sepostulo_predicciones = treeclassifier.predict(df_general_test[columnas_datos])

#prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
#print("Precision %.4f" % (prec * 100))

#del treeclassifier
#gc.collect()

### Random Forest

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
params = { 'n_estimators': 100, 'random_state': 23, 'min_samples_leaf': 3 }

rndforestclassifier = RandomForestClassifier(**params)
rndforestclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target].values.ravel())

sepostulo_predicciones = rndforestclassifier.predict(df_general_test[columnas_datos])

prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
print("Precision %.4f" % (prec * 100))

del rndforestclassifier
gc.collect()

### Extra Trees

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
#params = { 'n_estimators': 50, 'random_state': 23, 'min_samples_leaf': 3 }

#extratreesclassifier = RandomForestClassifier(**params)
#extratreesclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target].values.ravel())

#sepostulo_predicciones = extratreesclassifier.predict(df_general_test[columnas_datos])

#prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
#print("Precision %.4f" % (prec * 100))

#del extratreesclassifier
#gc.collect()

### XGBoost

In [51]:
from sklearn.externals import joblib

In [52]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
params = {'n_estimators': 100, 'learning_rate': 0.001 }#, 'max_depth': a, 'min_samples_split': 2, }

xgbclassifier = GradientBoostingClassifier(**params)
xgbclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target].values.ravel())

sepostulo_predicciones = xgbclassifier.predict(df_general_test[columnas_datos])
#sepostulo_predicciones_proba = xgbclassifier.predict_proba(df_general_test[columnas_datos])

prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
print("Precision %.4f" % (prec * 100))

joblib.dump(xgbclassifier, "xgboostclassifier.pkl")

del xgbclassifier
gc.collect()

Precision 99.8569


NameError: name 'clasificador' is not defined

In [53]:

prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
print("Precision %.4f" % (prec * 100))

Precision 99.8569


In [54]:

joblib.dump(xgbclassifier, "xgboostclassifier.pkl")

['xgboostclassifier.pkl']

### AdaBoost

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
#params = {'n_estimators': 100, 'learning_rate': 0.001, 'random_state': 29}

#adaboosclassifier = AdaBoostClassifier(**params)
#adaboosclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target].values.ravel())

#sepostulo_predicciones = adaboosclassifier.predict(df_general_test[columnas_datos])

#prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
#print("Precision %.4f" % (prec * 100))

#del adaboosclassifier
#gc.collect()

### SVM (RBF)

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
#params = {'C': 1.0, 'gamma': 'auto'}

#svcclassifier = SVC(**params)
#svcclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target])

#sepostulo_predicciones = svcclassifier.predict(df_general_test[columnas_datos])

#prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
#print("Precision %.4f" % (prec * 100))

### Red Neuronal (Multi-Layer Perceptron)

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
#params = {'solver': 'adam', 'hidden_layer_sizes': (50, 25) }#, 'alpha': 1e-5,}

#mlpclassifier = MLPClassifier(**params)
#mlpclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target].values.ravel())

#sepostulo_predicciones = mlpclassifier.predict(df_general_test[columnas_datos])

#prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones)
#print("Precision %.4f" % (prec * 100))

#del mlpclassifier
#gc.collect()

In [ ]:
#gc.collect()